# Try MWAS with covariates prepared in various ways

# Pick the regions we will test

In [1]:
library(data.table)

prev_hits <- fread("
Chr     pos        old_z       old_p
11   38247902        -27.1535308285104       2.30024742330298e-162
2    47933357        18.3327793004811        4.53147699327216e-75
7    1987910         10.1255085321387        4.25759742738181e-24
7    1987896         10.1255085321387        4.25759742738189e-24
7    1987797         10.0241523897721        1.19379483108027e-23
7    1987778         10.0105851568856        1.36940148731912e-23
12   2194742         -10.0072634920486       1.41615523554955e-23
")


In [2]:
#df <- fread("09.5-OUT_matched_SNP_meth_cov_chunked_EXPANSE_a2.csv")

In [3]:
df <- fread("09-OUT_matched_SNP_meth_cov_a2.csv")

## Try original code

In [4]:
###### model: learn elastic net model on training data 
######---------Input: trainX, trainY
######---------Return: selected features and coefficents

elastic.net <- function(trainX,trainY){
    if(nrow(trainX)!=length(trainY)){
            stop("Number of observations is differerent")
    } 

    # optimize alpha---mixing parameter  
    #a <- seq(0, 1, 0.1)
    a <- 0.5

    search <- foreach(ai = a, .combine = rbind) %dopar% {
        cv.fit <- cv.glmnet(
                        trainX,
                        trainY,
                        nfold = 5,
                        type.measure = "mse",
                        paralle = TRUE,
                        alpha = ai
                        )
        data.frame(
                        cvm = min(cv.fit$cvm),
                        lambda = cv.fit$lambda.min,
                        alpha = ai
                        )
        } 
    cv.opt <- search[search$cvm == min(search$cvm),] 

        # fit model by optimized alpha and lambda
        yfit = glmnet(
        trainX,
        trainY,
        lambda = cv.opt$lambda,
        alpha = cv.opt$alpha
                )       
        idf <- coef(yfit)
        idx <- which(idf != 0)
        selectf <- data.frame(
                features = idf@Dimnames[[1]][idx], 
                coefs = idf [idx]
        )
}

MWAS <- function(gwas, weight, geno){
        z <- gwas %*% weight
        z.cor <- cor(geno)
        se <- sqrt(weight %*%  z.cor %*%  weight)
        z <- z/se
        p=pnorm(abs(z),lower.tail=F)*2
        return(c(z, p))
}

In [5]:
df <- df[which(df$Chr == 7), ]

In [6]:
i <- 2

In [7]:
library("glmnet")
library("e1071")
library("doParallel")

set.seed(2018)
wind <- c(5000,10000)
# output directory
#outd <- "/dcl02/lieber/shan/shizhong/finemapping/GWAS/tags/scz3/mwas/chr22/1/"
outd <- "20-OUT_original_mwas_sanity_test/"

Loading required package: Matrix

Loaded glmnet 4.1-8

Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [8]:
suppressWarnings(library(bsseq))

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:Matrix’:

    expand, unname


The following objects are masked from ‘package:data.table’:

    first, second


The following object is masked from ‘package:utils’:

    findMatches


The

In [9]:
# old p
load("p1.rda")

In [10]:
p[1:10, 1:10]

1980077,0.9623693,0.9533113,0.9488137,0.9382598,0.9446401,0.9215182,0.9263652,0.9588217,0.9330597,0.9588301
1980101,0.9623933,0.9536637,0.9487929,0.9372716,0.9453945,0.9216584,0.9256115,0.9593576,0.9326077,0.9595451
1980129,0.9623260,0.9543107,0.9485887,0.9362757,0.9460740,0.9218622,0.9248481,0.9598699,0.9323198,0.9601973
1980136,0.9622831,0.9545096,0.9484938,0.9360728,0.9462013,0.9219193,0.9247144,0.9599708,0.9323138,0.9603281
1980145,0.9622125,0.9547853,0.9483458,0.9358404,0.9463407,0.9219962,0.9245785,0.9600847,0.9323456,0.9604767
1980179,0.9617977,0.9560054,0.9475364,0.9352418,0.9466390,0.9223291,0.9244150,0.9603628,0.9328418,0.9608582
1980183,0.9617343,0.9561653,0.9474163,0.9351995,0.9466520,0.9223756,0.9244306,0.9603806,0.9329368,0.9608897
1980193,0.9615631,0.9565781,0.9470942,0.9351188,0.9466655,0.9224991,0.9244996,0.9604121,0.9332054,0.9609580
1980205,0.9613345,0.9570958,0.9466676,0.9350679,0.9466476,0.9226612,0.9246379,0.9604260,0.9335838,0.9610206
1980239,0.9605591,0.9586691,0.9452337,0.9351803,0.9464118,0.9231993,0.9253325,0.9603334,0.9349537,0.9610909


In [11]:
BSsample$brnum

[1] "Br0836" "Br0845" "Br0848" "Br0863" "Br0914" "Br0948" "Br0949" "Br0963"
  [9] "Br0983" "Br0991" "Br0993" "Br1003" "Br1004" "Br1007" "Br1016" "Br1017"
 [17] "Br1021" "Br1023" "Br1030" "Br1033" "Br1034" "Br1039" "Br1040" "Br1050"
 [25] "Br1053" "Br1054" "Br1056" "Br1067" "Br1069" "Br1081" "Br1084" "Br1092"
 [33] "Br1093" "Br1102" "Br1109" "Br1113" "Br1118" "Br1122" "Br1134" "Br1135"
 [41] "Br1137" "Br1139" "Br1148" "Br1156" "Br1157" "Br1160" "Br1164" "Br1167"
 [49] "Br1183" "Br1185" "Br1187" "Br1189" "Br1191" "Br1193" "Br1203" "Br1204"
 [57] "Br1214" "Br1218" "Br1221" "Br1227" "Br1228" "Br1245" "Br1249" "Br1250"
 [65] "Br1263" "Br1265" "Br1271" "Br1275" "Br1276" "Br1279" "Br1280" "Br1281"
 [73] "Br1283" "Br1285" "Br1288" "Br1297" "Br1298" "Br1303" "Br1314" "Br1320"
 [81] "Br1321" "Br1324" "Br1325" "Br1326" "Br1339" "Br1340" "Br1342" "Br1346"
 [89] "Br1348" "Br1350" "Br1351" "Br1356" "Br1371" "Br1378" "Br1380" "Br1383"
 [97] "Br1384" "Br1389" "Br1390" "Br1399" "Br1408" "Br1410" "Br1412" "Br1416"
[105] "Br1420" "Br1424" "Br1436" "Br1441" "Br1442" "Br1443" "Br1446" "Br1447"
[113] "Br1448" "Br1453" "Br1455" "Br1459" "Br1464" "Br1474" "Br1494" "Br1497"
[121] "Br1498" "Br1499" "Br1503" "Br1504" "Br1507" "Br1509" "Br1517" "Br1522"
[129] "Br1525" "Br1527" "Br1529" "Br1531" "Br1536" "Br1538" "Br1539" "Br1548"
[137] "Br1550" "Br1552" "Br1556" "Br1558" "Br1565" "Br1568" "Br1571" "Br1573"
[145] "Br1576" "Br1578" "Br1579" "Br1580" "Br1585" "Br1588" "Br1590" "Br1594"
[153] "Br1602" "Br1604" "Br1605" "Br1610" "Br1611" "Br1613" "Br1615" "Br1624"
[161] "Br1637" "Br1644" "Br1646" "Br1647" "Br1649" "Br1654" "Br1657" "Br1663"
[169] "Br1664" "Br1665" "Br1669" "Br1670" "Br1672" "Br1674" "Br1676" "Br1678"
[177] "Br1682" "Br1688" "Br1693" "Br1695" "Br1700" "Br1703" "Br1708" "Br1722"
[185] "Br1735" "Br1738" "Br1740" "Br1750" "Br1753" "Br1756" "Br1761" "Br1764"
[193] "Br1832" "Br1840" "Br1842" "Br1856" "Br1865" "Br1883" "Br1898" "Br1905"
[201] "Br1912" "Br1918" "Br1927" "Br1941" "Br1946" "Br1958" "Br1960" "Br1980"
[209] "Br1987" "Br1988" "Br1989" "Br1994" "Br1995" "Br1998" "Br2001" "Br2003"
[217] "Br2009" "Br2020" "Br2038" "Br2044" "Br2046" "Br2052" "Br2053" "Br2064"
[225] "Br2074" "Br2075" "Br2076" "Br2084" "Br2242" "Br2257" "Br2267" "Br2271"
[233] "Br2285" "Br2287" "Br2288" "Br2292" "Br2295" "Br2298" "Br2337" "Br2345"
[241] "Br2346" "Br2351" "Br2353" "Br2365" "Br2377" "Br2388" "Br2421" "Br2423"
[249] "Br2425" "Br2454" "Br2464" "Br2470" "Br2476" "Br2481" "Br2494" "Br2509"
[257] "Br2513" "Br2534" "Br2539" "Br2542" "Br2543" "Br2573" "Br2582" "Br2585"
[265] "Br2587" "Br5005" "Br5062" "Br5066" "Br5073" "Br5154" "Br5161" "Br5165"
[273] "Br5179" "Br5182" "Br5200" "Br5205" "Br5207" "Br5212" "Br5224" "Br5228"
[281] "Br5234" "Br5239" "Br5242" "Br5259" "Br5262" "Br5264" "Br5280" "Br5282"
[289] "Br5287" "Br5292" "Br5295" "Br5315" "Br5319" "Br5323" "Br5324" "Br5330"
[297] "Br5347" "Br5359" "Br5373" "Br5398" "Br5422" "Br5426" "Br5460" "Br5467"
[305] "Br5475" "Br5488" "Br5584" "Br5590"

In [12]:
dim(p)

[1] 330 308

In [13]:
length(BSsample$brnum)

[1] 308

In [14]:
colnames(p) <- BSsample$brnum

In [15]:
head(p)

,Br0836,Br0845,Br0848,Br0863,Br0914,Br0948,Br0949,Br0963,Br0983,Br0991,⋯,Br5373,Br5398,Br5422,Br5426,Br5460,Br5467,Br5475,Br5488,Br5584,Br5590
1980077,0.9623693,0.9533113,0.9488137,0.9382598,0.9446401,0.9215182,0.9263652,0.9588217,0.9330597,0.9588301,⋯,0.9562401,0.9197686,0.9143680,0.9173880,0.9450752,0.9593447,0.9398029,0.9671064,0.9308173,0.9355940
1980101,0.9623933,0.9536637,0.9487929,0.9372716,0.9453945,0.9216584,0.9256115,0.9593576,0.9326077,0.9595451,⋯,0.9574444,0.9190366,0.9191129,0.9165164,0.9434155,0.9596104,0.9385970,0.9678762,0.9307398,0.9377196
1980129,0.9623260,0.9543107,0.9485887,0.9362757,0.9460740,0.9218622,0.9248481,0.9598699,0.9323198,0.9601973,⋯,0.9586133,0.9183214,0.9240780,0.9158721,0.9414392,0.9599566,0.9372357,0.9686038,0.9307874,0.9394777
1980136,0.9622831,0.9545096,0.9484938,0.9360728,0.9462013,0.9219193,0.9247144,0.9599708,0.9323138,0.9603281,⋯,0.9588788,0.9181811,0.9252248,0.9157864,0.9409121,0.9600495,0.9369054,0.9687640,0.9308332,0.9398197
1980145,0.9622125,0.9547853,0.9483458,0.9358404,0.9463407,0.9219962,0.9245785,0.9600847,0.9323456,0.9604767,⋯,0.9592064,0.9180243,0.9266443,0.9157199,0.9402138,0.9601728,0.9364877,0.9689584,0.9309120,0.9402076
1980179,0.9617977,0.9560054,0.9475364,0.9352418,0.9466390,0.9223291,0.9244150,0.9603628,0.9328418,0.9608582,⋯,0.9603121,0.9176651,0.9314597,0.9158901,0.9373834,0.9606790,0.9349929,0.9695837,0.9313985,0.9411866


In [16]:
# load data for mwas
# load("./rda/caudate_mwas_data_chr22.rda")
load(df$methylation_data[i])

pnew <- getMeth(BSobj2)

In [17]:
rownames(pnew) <- start(BSobj2)

In [18]:
pnew[1:10, 1:10]

<10 x 10> DelayedMatrix object of type "double":
           [,1]      [,2]      [,3] ...      [,9]     [,10]
49742 0.8712488 0.7818928 0.8298454   . 0.7811685 0.8457106
49782 0.8694834 0.7819481 0.8280379   . 0.7832709 0.8456391
49895 0.8639242 0.7833570 0.8240252   . 0.7893901 0.8458250
50060 0.8534662 0.7868178 0.8206575   . 0.7980355 0.8465550
50097 0.8507598 0.7875764 0.8202329   . 0.7998109 0.8465572
50146 0.8469822 0.7885745 0.8198201   . 0.8020215 0.8464591
50157 0.8461047 0.7887977 0.8197485   . 0.8024929 0.8464220
50163 0.8456215 0.7889193 0.8197125   . 0.8027459 0.8463996
50173 0.8448093 0.7891219 0.8196569   . 0.8031609 0.8463586
50244 0.8387986 0.7905562 0.8194082   . 0.8058465 0.8459486

In [ ]:
sites_to_test <- which(start(BSobj2) >= (1987910 - 500) & start(BSobj2) <= (1987910 + 500))
p <- p[sites_to_test, ]

# candidate cg
cg <- as.numeric(rownames(p))

# regress out covariates
#load("covs_for_meqtl.rda")

In [10]:
covs <- fread(df$cov_file2[i])
covs <- t(covs)
colnames(covs) <- covs[1, ]
covs <- covs[2:nrow(covs), ]
# transpose so we have same orientation as original code

In [11]:
mat <- match(BSobj2$brnum,colnames(covs)) 
covs <- t(covs[,mat])
p.residual=matrix(0,dim(p)[1],dim(p)[2])

In [12]:
rownames(covs)[is.na(covs[, 'genoPC1'])] <- BSobj2$brnum[is.na(covs[, 'genoPC1'])]

In [13]:
colnames(p.residual) <- BSobj2$brnum

In [14]:
covs <- as.data.frame(covs)
# Convert all columns except Dx and Sex from character to numeric
cols_to_convert <- setdiff(names(covs), c("Dx", "Sex"))

for (col in cols_to_convert) {
  covs[[col]] <- as.numeric(covs[[col]])
}

# Print the modified data frame to check the conversion
#print(dat)


In [15]:
for (i in 1:dim(p)[1]) { # For each methylation site
    dat <- as.data.frame(cbind(y = p[i,], covs))
    
    # Check for rows with NAs (the ones for which we don't have covariate data)
    valid_rows <- complete.cases(dat)
    
    if (sum(valid_rows) > 0) {
        dat_valid <- dat[valid_rows,]
        model.res <- lm(y ~ ., data = dat_valid)
        
        # Store residuals in the corresponding positions
        p.residual[i, valid_rows] <- resid(model.res)
    }
}


# for(i in 1:dim(p)[1]){ # foro each methylation site
#         dat <- as.data.frame(cbind(p[i,],covs))
#         colnames(dat) <- c("y",paste0("x",1:ncol(covs)))
#         model.res <- lm(reformulate(paste0("x",1:ncol(covs)), "y"),dat)
#         p.residual[i,] = resid(model.res) 
# }

In [16]:
load("p1.rda")

In [17]:
wind <- 10000

In [18]:
# built predition models
idx.ea <- BSobj2$race == "CAUC"

In [19]:
snp2 <- snp2[, colnames(snp2) %in% colnames(p.residual)]

In [20]:
for(k in 1:length(wind)){
    models.ea <- c()
    models.all <- c()
    for(i in 1:length(cg)){
            cat(i,"\n")
            range1 <- ifelse(cg[i] - wind[k] > 0,cg[i] - wind[k],0)
            range2 <- cg[i] + wind[k]
            idx <- map2$POS > range1 & map2$POS < range2
            # go to next cg if no snps within window
            if(sum(idx) <= 1){
                    next
            }
            geno <- snp2[idx,]
            rownames(geno) <- map2$POS[idx]
            trainX <- t(geno)
            trainY <- p.residual[i,]
            fit <- tryCatch(
                    elastic.net(trainX,trainY),
                    error = function(e) {return ("err")})
            if(!is.data.frame(fit)){
                if(fit == "err"){
                    next
                }
            }

            fit$cg <- cg[i]
            models.all <- rbind(models.all,fit)
            # EA only
            trainX <- trainX[idx.ea,]
            if(sum(apply(trainX,2,var)!=0) <= 1){
                    next
            }
            trainY <- trainY[idx.ea]
            fit <- tryCatch(
                    elastic.net(trainX,trainY),
                    error = function(e) {return ("err")})
            if(!is.data.frame(fit)){
                if(fit == "err"){
                    next
                }
            }
            fit$cg <- cg[i]
            models.ea <- rbind(models.ea,fit)
    }
}

1 


Warning message:
“executing %dopar% sequentially: no parallel backend registered”


2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 


In [21]:
models.ea <- models.ea[models.ea[,1] != "(Intercept)",]
models.all <- models.all[models.all[,1] != "(Intercept)",]

In [22]:
if(!dir.exists(outd)) dir.create(outd)

In [23]:
# mwas by models of all samples
cg2 <- unique(models.all$cg)
mwas.all <- matrix(0,nrow=length(cg2),ncol=2)
for(i in 1:length(cg2)){
        pos <- models.all[models.all$cg == cg2[i],1]
        gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
        weight <- models.all[models.all$cg == cg2[i],2]
        geno <- snp.1kg.eur2[match(pos,map.1kg.eur2$POS),]
        mwas.all[i,] <- MWAS(gwas, weight, t(geno))
}
rownames(mwas.all) <- cg2
colnames(mwas.all) <- c("z","p")

# mwas by models of EA samples
cg2 <- unique(models.ea$cg)
mwas.ea <- matrix(0,nrow=length(cg2),ncol=2)
for(i in 1:length(cg2)){
        pos <- models.ea[models.ea$cg == cg2[i],1]
        gwas <- snp.gwas2$z[is.element(snp.gwas2$pos_hg38, pos)]
        weight <- models.ea[models.ea$cg == cg2[i],2]
        geno <- snp.1kg.eur2[match(pos,map.1kg.eur2$POS),]
        mwas.ea[i,] <- MWAS(gwas, weight, t(geno))
}
rownames(mwas.ea) <- cg2
colnames(mwas.ea) <- c("z","p")

# output models and mwas results
outf <- paste0(outd,"/models-cov2.all.wind.",wind[k])
write.csv(models.all,outf)
outf <- paste0(outd,"/models-cov2.ea.wind.",wind[k])
write.csv(models.ea,outf)
outf <- paste0(outd,"/mwas-cov2.all.wind.",wind[k])
write.csv(mwas.all,outf)
outf <- paste0(outd,"/mwas-cov2.ea.wind.",wind[k])
write.csv(mwas.ea,outf)